In [ ]:
from urbansim.models.dcm import MNLDiscreteChoiceModel

import os
import numpy as np
import pandas as pd

import orca
from urbansim.utils import misc

import dataset
import variables
import models
import utils

import yaml

orca.run(["build_networks"])
orca.run(["neighborhood_vars"])

### Estimate HLCM from config

In [ ]:
hlcm1_model = MNLDiscreteChoiceModel.from_yaml(str_or_buffer=misc.config('hlcm/hlcm1.yaml'))

hh = orca.get_table('households').to_frame(hlcm1_model.columns_used())
b = orca.get_table('buildings').to_frame(hlcm1_model.columns_used())

hh = hh[hh.income_quartile == 1]
b = b.fillna(0)
hh = hh.fillna(0)

print hlcm1_model.fit(hh, b, hh[hlcm1_model.choice_column])

hlcm1_model.fit_parameters

## Hedonic Estimation

In [ ]:
orca.run(["rsh_estimate"])

In [ ]:
orca.run(["nrh_estimate"])#

# adjust hedonic model specification and update yamls based on P values

In [ ]:
import math
import yaml
conf = "./configs/rsh"

In [ ]:
with open(conf + ".yaml", 'r') as stream:
    dict_spec = yaml.load(stream)

In [ ]:
droplist =['building_id', 'building_type_id', 'non_residential_sqft', 'parcel_id', 
           'improvement_value', 'sqft_price_nonres', 'sqft_price_res', 'city_id', 
           'parcels_max_far',  'semmcd','parcels_county_id', 'nodeid_drv', 'zones_taz',
           'parcels_max_dua', 'parcels_allowed',  'parcels_school_id',  'parcels_land_cost', 
           'y', 'building_type_id_is_99', 'x', 'parcels_sev_value', 
           'zones_acres', 'general_type', 'parcels_land_use_type_id', 'zones_tazce10_n', 
           'parcels_census_bg_id', 'large_area_id', 'parcels_max_height']

In [ ]:
total=0
for m in dict_spec['models'].keys():
    print 'model: ', m
    
    if not(m in [42,43,341,541,9341,9941,11541,12541,14741,16141,42.0,41.0]):
        mdl = dict_spec['models'][m]['model_expression'].replace("  "," ")
        for item in droplist:
            mdl  = mdl.replace(' + ' + item + ' + ',' + ')

        rmlist = []
        for k, v in dict_spec['models'][m]['fit_parameters']['P-Values'].iteritems():
            if (v > 0.05) or (math.isnan(v)):
                mdl  = mdl.replace('+ ' + k + ' +',' +')
                rmlist.append(k)
                total += 1
        print rmlist

        dict_spec['models'][m]['model_expression'] = mdl
print "total bad vars before changes: ", total    
    

In [ ]:
with open(conf + "_new.yaml" , 'w') as outfile:
    yaml.dump(dict_spec, outfile, default_flow_style=False, indent=4)

## LCM Estimation

In [ ]:
orca.run(["elcm_estimate"])

# data inspection

In [ ]:
# EXTRACT BUILDING data for external estimation

#buildings=orca.get_table('buildings').to_frame()
#buildings.dropna(axis=1, how='any', inplace=True)
#store=pd.HDFStore('hedonic.h5')
#buildings.to_hdf(store, 'buildings', complib='blosc', complevel=9)
#store.close()


In [ ]:
df=orca.get_table('nodes_walk').to_frame()

In [ ]:
df[df.isnull().any(axis=1)]

In [ ]:
df=orca.get_table('zones').to_frame()